In [1]:
import os
import time

import requests
from PIL import Image
from tqdm.auto import tqdm

/Users/maxwoolf/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
folder_name = "pokemon_images.nosync"
size = 224  # required for the nomic vision model

graphql_query = """
{
  pokemon_v2_pokemon(order_by: {id: asc}) {
    id
  }
}
"""

image_url = (
    "https://raw.githubusercontent.com/PokeAPI/sprites/master/"
    "sprites/pokemon/other/official-artwork/{0}.png"
)

In [10]:
r = requests.post(
    "https://beta.pokeapi.co/graphql/v1beta",
    json={
        "query": graphql_query,
    },
)

pokemon = r.json()["data"]["pokemon_v2_pokemon"]
pokemon[-10:]

[{'id': 10268},
 {'id': 10269},
 {'id': 10270},
 {'id': 10271},
 {'id': 10272},
 {'id': 10273},
 {'id': 10274},
 {'id': 10275},
 {'id': 10276},
 {'id': 10277}]

In [19]:
print(f"Saving Pokemon images to /{folder_name} .")

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
for p in tqdm(pokemon):
    p_id = p["id"]
    filename = f"{p_id}.png"
    if not os.path.exists(os.path.join(folder_name, filename)):
        try:
            img = Image.open(requests.get(image_url.format(p_id), stream=True).raw)
            img = img.resize((size, size), Image.Resampling.LANCZOS)

            # Composite transparent PNG onto white background
            # https://stackoverflow.com/a/9459208
            bg = Image.new("RGB", (size, size), (255, 255, 255))
            bg.paste(img, mask=img.split()[3])
            bg.save(os.path.join(folder_name, filename))
            time.sleep(0.5)
        except Image.UnidentifiedImageError:
            print(f"No image for ID {p_id}")

Saving Pokemon images to /pokemon_images.nosync .


 90%|████████▉ | 1170/1302 [00:00<00:00, 8756.16it/s]

No image for ID 10143
No image for ID 10145


 99%|█████████▉| 1286/1302 [01:16<00:04,  3.90it/s]  

No image for ID 10264
No image for ID 10265
No image for ID 10266
No image for ID 10267
No image for ID 10268
No image for ID 10269
No image for ID 10270
No image for ID 10271


100%|██████████| 1302/1302 [01:24<00:00, 15.45it/s]
